In [1]:
try:
    import openmdao.api as om
except ImportError:
    !pip install openmdao[notebooks]
import openmdao.api as om

In [6]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [7]:
import openmdao.api as om


class Paraboloid(om.ExplicitComponent):
    """
    Evaluates the equation f(x,y) = (x-3)^2 + xy + (y+4)^2 - 3.
    """

    def setup(self):
        self.add_input('x', val=0.0)
        self.add_input('y', val=0.0)

        self.add_output('f_xy', val=0.0)

    def setup_partials(self):
        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')

    def compute(self, inputs, outputs):
        """
        f(x,y) = (x-3)^2 + xy + (y+4)^2 - 3

        Minimum at: x = 6.6667; y = -7.3333
        """
        x = inputs['x']
        y = inputs['y']

        outputs['f_xy'] = (x - 3.0)**2 + x * y + (y + 4.0)**2 - 3.0


if __name__ == "__main__":

    # build the model
    prob = om.Problem()
    prob.model.add_subsystem('parab', Paraboloid(), promotes_inputs=['x', 'y'])

    # define the component whose output will be constrained
    prob.model.add_subsystem('const', om.ExecComp('g = x + y'), promotes_inputs=['x', 'y'])

    # Design variables 'x' and 'y' span components, so we need to provide a common initial
    # value for them.
    prob.model.set_input_defaults('x', 3.0)
    prob.model.set_input_defaults('y', -4.0)

    # setup the optimization
    prob.driver = om.ScipyOptimizeDriver()
    prob.driver.options['optimizer'] = 'COBYLA'

    prob.model.add_design_var('x', lower=-50, upper=50)
    prob.model.add_design_var('y', lower=-50, upper=50)
    prob.model.add_objective('parab.f_xy')

    # to add the constraint to the model
    prob.model.add_constraint('const.g', lower=0, upper=10.)

    prob.setup()

In [8]:
from ipywidgets import Button, Label, FloatText, HBox, VBox, Text, Output


def om_interact(prob, vars=None):
    rows = []
    meta = prob.model.get_io_metadata()
    meta = {opts['prom_name']: opts for (_, opts) in prob.model.get_io_metadata().items()}
    output = Output()

    @output.capture()
    def handle_value_change(change):
        print(change)
        val = change['new']
        target = change['owner'].description
        prob.set_val(target, val)
        print(f'set {target} to {val}')

    for varname, options in meta.items():
        print(varname)
        if varname in vars:
            desc = options['desc']
            units = options['units']
            val = prob.get_val(varname, units=units)
            input_box = FloatText(val, description=varname)
            input_box.observe(handle_value_change, names='value')
            row = [Label(varname), input_box, Label(options['units'])]
            rows.append(HBox(row))

    @output.capture()
    def run_handler(change):
        output.clear_output()
        if change.description == 'Run driver':
            print('Running driver')
            prob.run_driver()
            prob.list_problem_vars()
        elif change.description == 'Run model':
            print('Running model')
            prob.run_model()
            prob.model.list_outputs()
        else:
            raise ValueError('Unhandled button: ', change.description)

    run_driver_button = Button(description='Run driver')
    run_driver_button.on_click(run_handler)

    run_model_button = Button(description='Run model')
    run_model_button.on_click(run_handler)

    rows.append(HBox([run_model_button, run_driver_button]))

    rows.append(output)

    return VBox(rows)

om_interact(prob, vars=['x', 'y'])

x
y
parab.f_xy
const.g
_auto_ivc.v0
_auto_ivc.v1


In [5]:
prob.list_problem_vars()

----------------
Design Variables
----------------
name  val           size  
----  ------------  ---- 
x     [6.9999987]   1     
y     [-6.9999987]  1     

-----------
Constraints
-----------
name     val           size  
-------  ------------  ---- 
const.g  [0.]          1     
x        [6.9999987]   1     
y        [-6.9999987]  1     

----------
Objectives
----------
name        val     size  
----------  ------  ---- 
parab.f_xy  [-27.]  1     

